In [31]:
import pandas as pd

df = pd.read_csv('parsed_smell_modified.csv', header=None, names=['AST'])
df

,AST
0,\n\t ClassDeclaration\n\t\t BaseList\n\t\t\t S...
1,\n\t\t FieldDeclaration\n\t\t\t VariableDeclar...
2,\n\t\t ConstructorDeclaration\n\t\t\t Paramete...
3,\n\t\t\t\t\t\t LocalDeclarationStatement\n\t\t...
4,\n\t\t MethodDeclaration\n\t\t\t PredefinedTyp...
5,\n\t\t\t\t Parameter\n\t\t\t\t\t PredefinedType
6,\n\t\t\t\t ExpressionStatement\n\t\t\t\t\t Inv...
7,\n\t ClassDeclaration\n\t\t BaseList\n\t\t\t S...
8,\n\t\t FieldDeclaration\n\t\t\t VariableDeclar...
9,\n\t\t\t\t\t\t LocalDeclarationStatement\n\t\t...


In [32]:
ast_list = df['AST'].tolist()
preprocessed_ast_list = [ast.strip() for ast in ast_list]

In [33]:
from sentence_transformers import SentenceTransformer, util
import torch


In [34]:
model = SentenceTransformer('bert-base-nli-mean-tokens')


In [35]:
ast_embeddings = model.encode(preprocessed_ast_list, convert_to_tensor=True)
save_path = 'smell_embeddings.pt'
torch.save(ast_embeddings, save_path)


In [36]:
loaded_ast_embeddings = torch.load(save_path)
loaded_ast_embeddings 

tensor([[-0.0950,  0.6250,  0.5913,  ..., -0.2796, -0.2381,  0.4071],
        [-0.2020,  0.1644,  1.1535,  ..., -0.3008, -0.0297,  0.5221],
        [-0.0143, -0.2363,  1.6680,  ..., -0.3705, -0.5433, -0.1417],
        ...,
        [-0.4625,  0.5366,  0.8137,  ..., -0.5727, -0.8327,  0.3506],
        [ 0.0656, -0.5847,  1.4588,  ..., -0.5001, -0.2873,  0.0663],
        [-0.4386,  0.4322,  0.7852,  ..., -0.5318, -0.3987,  0.7678]])

In [37]:
num_embeddings = loaded_ast_embeddings.shape[0]
print(f"Number of saved embeddings: {num_embeddings}")

Number of saved embeddings: 24


In [38]:
# Example: Compute similarity between the first and second ASTs
similarity_score = util.cos_sim(ast_embeddings[6], ast_embeddings[5])
print(f"Similarity score between AST 1 and AST 2: {similarity_score}")

Similarity score between AST 1 and AST 2: tensor([[0.5420]])


In [39]:
import numpy as np

num_embeddings = len(ast_embeddings)

for ast_index in range(num_embeddings):
    similarity_scores = np.zeros(num_embeddings)

    for i in range(num_embeddings):
        if i != ast_index:
            similarity_scores[i] = util.cos_sim(ast_embeddings[ast_index], ast_embeddings[i])

    # Sort the similarity scores in descending order
    sorted_indices = np.argsort(similarity_scores)[::-1]

    # List the similarity scores for the specific AST in descending order
    print(f"Ast {ast_index}:")
    for idx in sorted_indices:
        if idx != ast_index:
            score = similarity_scores[idx]
            print(f"AST {idx}  {score:.2f}")
    print()

Ast 0:
AST 10  1.00
AST 7  0.98
AST 1  0.85
AST 18  0.85
AST 3  0.81
AST 17  0.81
AST 21  0.81
AST 11  0.80
AST 13  0.80
AST 6  0.79
AST 2  0.77
AST 23  0.77
AST 19  0.75
AST 12  0.75
AST 8  0.74
AST 16  0.73
AST 20  0.73
AST 22  0.72
AST 5  0.66
AST 4  0.65
AST 9  0.64
AST 14  0.64
AST 15  0.64

Ast 1:
AST 17  0.92
AST 3  0.92
AST 8  0.91
AST 23  0.91
AST 7  0.87
AST 15  0.86
AST 14  0.86
AST 10  0.85
AST 0  0.85
AST 13  0.85
AST 18  0.83
AST 21  0.82
AST 6  0.81
AST 11  0.81
AST 20  0.81
AST 12  0.81
AST 2  0.79
AST 19  0.78
AST 9  0.76
AST 22  0.74
AST 5  0.72
AST 16  0.69
AST 4  0.69

Ast 2:
AST 12  0.94
AST 11  0.92
AST 22  0.90
AST 8  0.83
AST 5  0.83
AST 7  0.80
AST 3  0.80
AST 16  0.80
AST 1  0.79
AST 17  0.79
AST 18  0.78
AST 10  0.77
AST 0  0.77
AST 21  0.76
AST 4  0.76
AST 13  0.76
AST 23  0.75
AST 6  0.75
AST 20  0.73
AST 19  0.72
AST 14  0.71
AST 15  0.71
AST 9  0.66

Ast 3:
AST 17  0.98
AST 23  0.93
AST 1  0.92
AST 18  0.91
AST 21  0.91
AST 13  0.90
AST 6  0.89
AST 20  0.

In [42]:
import numpy as np

num_embeddings = len(ast_embeddings)

# Choose the index of the specific AST for comparison
ast_index = 0  # Replace with the desired AST index

similarity_scores = np.zeros(num_embeddings)

for i in range(num_embeddings):
    if i != ast_index:
        similarity_scores[i] = util.cos_sim(ast_embeddings[ast_index], ast_embeddings[i])

# Sort the similarity scores in descending order
sorted_indices = np.argsort(similarity_scores)[::-1]
sorted_scores = similarity_scores[sorted_indices]

# List the similarity scores for the specific AST in descending order
print(f"Ast {ast_index}:")
for i in range(num_embeddings - 1):
    idx = sorted_indices[i]
    score = sorted_scores[i]
    print(f"AST {idx}  {score:.2f}")

Ast 0:
AST 10  1.00
AST 7  0.98
AST 1  0.85
AST 18  0.85
AST 3  0.81
AST 17  0.81
AST 21  0.81
AST 11  0.80
AST 13  0.80
AST 6  0.79
AST 2  0.77
AST 23  0.77
AST 19  0.75
AST 12  0.75
AST 8  0.74
AST 16  0.73
AST 20  0.73
AST 22  0.72
AST 5  0.66
AST 4  0.65
AST 9  0.64
AST 14  0.64
AST 15  0.64


In [ ]:
rows_to_print = [0, 10, 1]
columns_to_print = ['AST']
print(df.loc[rows_to_print, columns_to_print].to_string(index=False))